
# Quantum Harmonic Oscillator with Perturbation Theory

This notebook explores the **Quantum Harmonic Oscillator (QHO)**, its unperturbed solutions, 
and the effect of a small perturbation using **time-independent perturbation theory**.  

We follow the same structure as the *potential well* notebook, but applied to the harmonic oscillator.



## 1. Unperturbed Harmonic Oscillator

In classical mechanics, the harmonic oscillator obeys Hooke's law:  

$$
m \frac{d^2x}{dt^2} = -kx, \qquad V(x) = \tfrac{1}{2} m \omega^2 x^2, \quad \omega = \sqrt{\tfrac{k}{m}}.
$$

In quantum mechanics, the Schrödinger equation is:  

$$
-\frac{\hbar^2}{2m}\frac{d^2 f}{dx^2} + \tfrac{1}{2} m \omega^2 x^2 f(x) = E f(x).
$$

Introduce the dimensionless variable  

$$
\xi = \sqrt{\tfrac{m \omega}{\hbar}} x,
$$

so the equation becomes  

$$
\frac{d^2 f}{d\xi^2} = (\xi^2 - K) f, \qquad K = \frac{2E}{\hbar \omega}.
$$

For large $\xi$, solutions behave like $e^{-\xi^2/2}$ (normalizable) or $e^{+\xi^2/2}$ (divergent).  
Thus we set

$$
f(\xi) = \eta(\xi) e^{-\xi^2/2},
$$

with $\eta(\xi)$ a polynomial. Requiring termination of the power series leads to quantized energies:

$$
E_n = \hbar \omega \Bigl(n + \tfrac{1}{2}\Bigr), \qquad n = 0, 1, 2, \dots
$$

and normalized eigenfunctions:

$$
f_n(x) = \Bigl(\frac{m\omega}{\pi\hbar}\Bigr)^{1/4}\frac{1}{\sqrt{2^n n!}} H_n(\xi) e^{-\xi^2/2},
$$

where $H_n(\xi)$ are Hermite polynomials.



## 2. Perturbation Theory

We now add a small perturbation of the form  

$$
V'(x) = \varepsilon x.
$$

The **first-order correction** is

$$
E_n^{(1)} = \langle f_n | \varepsilon x | f_n \rangle.
$$

Since $x$ only couples $n \to n \pm 1$, this expectation value vanishes by orthogonality:

$$
E_n^{(1)} = 0.
$$

The **second-order correction** is

$$
E_n^{(2)} = \sum_{m \neq n} \frac{|\langle f_m | \varepsilon x | f_n \rangle|^2}{E_n^{(0)} - E_m^{(0)}}.
$$

Only $m=n\pm 1$ contribute, giving

$$
E_n^{(2)} = -\frac{\varepsilon^2}{2 m \omega^2},
$$

which is independent of $n$ (a Stark-like quadratic shift).


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from scipy.special import hermite, factorial

# Constants
hbar = 1.0
m = 1.0
omega = 1.0


In [ ]:

def E_n(n, m=m, omega=omega, hbar=hbar):
    return hbar * omega * (n + 0.5)

def f_n(n, x, m=m, omega=omega, hbar=hbar):
    xi = np.sqrt(m*omega/hbar) * x
    Hn = hermite(n)(xi)
    norm = (m*omega/(np.pi*hbar))**0.25 / np.sqrt(2.0**n * factorial(n))
    return norm * Hn * np.exp(-xi**2/2)


In [ ]:

def first_order_correction(n, epsilon, m=m, omega=omega, hbar=hbar):
    # Vanishes for this perturbation
    return 0.0

def second_order_correction(n, epsilon, m=m, omega=omega, hbar=hbar):
    En0 = E_n(n, m, omega, hbar)
    corr = 0.0
    
    # n+1 state
    if n+1 >= 0:
        x_nm = np.sqrt(hbar/(2*m*omega)) * np.sqrt(n+1)
        corr += (epsilon * x_nm)**2 / (En0 - E_n(n+1, m, omega, hbar))
    
    # n-1 state
    if n-1 >= 0:
        x_nm = np.sqrt(hbar/(2*m*omega)) * np.sqrt(n)
        corr += (epsilon * x_nm)**2 / (En0 - E_n(n-1, m, omega, hbar))
    
    return corr


In [ ]:

# Parameters
epsilon = 0.5
n = 2
a = np.sqrt(hbar/(m*omega))
x = np.linspace(-4*a, 4*a, 400)

# Energy corrections
E0 = E_n(n)
E1 = first_order_correction(n, epsilon)
E2 = second_order_correction(n, epsilon)

# Wavefunctions
psi0 = f_n(n, x)
psi1 = 0 # first order correction vanishes
psi_total = psi0 # since only 2nd order shifts energy

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14,6))

# Energy levels
axes[0].hlines(E0, 0.5, 1.5, color='blue', label='Unperturbed $E_n^{(0)}$')
axes[0].hlines(E0+E1, 1.5, 2.5, color='red', label='1st order')
axes[0].hlines(E0+E1+E2, 2.5, 3.5, color='green', label='2nd order')
axes[0].set_title("Energy levels")
axes[0].set_ylabel("Energy")
axes[0].set_xticks([])
axes[0].legend()

# Wavefunctions
axes[1].plot(x, psi0, label='Unperturbed $\\psi_n^{(0)}$', color='blue')
axes[1].plot(x, psi_total, label='Perturbed $\\psi_n$', color='green')
axes[1].set_title("Wavefunctions")
axes[1].set_xlabel("x")
axes[1].set_ylabel("$\\psi(x)$")
axes[1].legend()

plt.show()
